In [1]:
import pandas as pd
import requests
url = 'https://team-plan.glitch.me/api'
res = requests.get(url)
json = res.json()
csv = pd.DataFrame(json['csv'])
values = pd.DataFrame(json['values'])

In [2]:
# import ipdb
objectives = pd.Series(csv.loc[csv.team==''].text).rename('objective')
teams = pd.Series(csv.team.unique()[1:]).rename('team')
tasks = csv.loc[csv.team!=''].drop(['_id', '__v'], axis=1)
tasks.loc[:, 'team'] = list(map(lambda x: pd.Index(teams).get_loc(x), tasks.team.tolist()))
tasks.loc[:, 'objective'] = list(map(lambda x: pd.Index(objectives).get_loc(x), tasks.objective.tolist()))
tasks = tasks.assign(complete = 'FALSE')

In [3]:
dir = '.data/'
objectives.to_csv(dir + 'objectives.csv', header=True)
teams.to_csv(dir + 'teams.csv', header=True)
tasks.to_csv(dir + 'tasks.csv')

In [4]:
values.drop(['_id', '__v'], axis=1).to_csv(dir + 'values.csv')

In [5]:
import sqlite3
import ipdb
conn = sqlite3.connect('.data/main.db')
c = conn.cursor()
# objectives, teams, tasks, values
# INTEGER PRIMARY KEY AUTOINCREMENT 

# '''CREATE TABLE stocks
#              (date text, trans text, symbol text, qty real, price real)''''''
drop_objectives = """
    drop table objectives;
"""

drop_teams = """
    drop table teams;
"""

drop_tasks = """
    drop table tasks;
"""

drop_values = """
    drop table library_values;
""" 

create_objectives_sql = """
    CREATE TABLE objectives
        (id INTEGER PRIMARY KEY AUTOINCREMENT, objective text);
"""
create_teams_sql = """
    CREATE TABLE teams
        (id INTEGER PRIMARY KEY AUTOINCREMENT, team text);
"""
create_tasks_sql = """
    CREATE TABLE tasks
        (id INTEGER PRIMARY KEY AUTOINCREMENT, task text, team text, objective text, complete text);
"""
create_values_sql = """
    CREATE TABLE library_values
        (id INTEGER PRIMARY KEY AUTOINCREMENT, value text);
"""
sql_list = [drop_objectives, drop_teams, drop_tasks, drop_values, create_objectives_sql, create_teams_sql, create_tasks_sql, create_values_sql]
for s in sql_list:
    c.execute(s)
    conn.commit()

In [6]:
# c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)
conn = sqlite3.connect('.data/main.db')
c = conn.cursor()
c.executemany('INSERT INTO objectives values (?, ?);', list(objectives.items()))
conn.commit()


In [7]:
c.executemany('INSERT INTO teams values (?, ?);', list(teams.items()))
conn.commit()

In [8]:
# INSERT INTO COMPANY (NAME,AGE,ADDRESS,SALARY)
# VALUES ( 'Paul', 32, 'California', 20000.00 );
import sqlite3
conn = sqlite3.connect('.data/main.db')
c = conn.cursor()
task_list = []
for (idx, task) in enumerate(tasks.values.tolist()):
    task.insert(0, idx)
    task_list.append(task)

c.executemany('INSERT INTO tasks values (?, ?, ?, ?, ?);', task_list)
conn.commit()

In [9]:
c.executemany('INSERT INTO library_values values (?, ?);', list(values.text.items()))
conn.commit()